<a href="https://colab.research.google.com/github/inesbsilveira/hummingbirds/blob/main/Risk_classification/all_risks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap
!pip install geojson

In [3]:

import os
import csv
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, mapping
from shapely.validation import make_valid
import geojson
import ee
import geemap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns
import ee
import datetime

In [4]:

my_project = 'ee-ineshummingbirds'
ee.Authenticate()
ee.Initialize(project= my_project)


In [5]:
# Define the period of time for temperature and precipitation
start_date = '2024-01-01'
end_date = '2024-12-31'



In [9]:

# if the input is a shapefile
input_shp = "original_calao_shp.shp"
gdf = gpd.read_file(input_shp).to_crs('EPSG:4326')
region = geemap.geopandas_to_ee(gdf)


In [18]:
#FUNCTIONS
# Count the number of days with at least one pixel exceeding the threshold
def count_hot_days(image):
    mask = image.reduceRegion(
        reducer=ee.Reducer.anyNonZero(),
        geometry=region,
        scale=1000,
        maxPixels=1e8
    )
    is_above = ee.Algorithms.If(mask.get('temperature_2m'), 1, 0)
    return ee.Feature(None, {'date': image.get('system:time_start'), 'day_above_32': is_above})

def count_flood_events(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)

    yearly_floods = (gfd.filterDate(start_date, end_date)
                      .select('flooded')
                      .map(lambda img: img.gt(0).And(jrc.Not()))
                      .sum()
                      .gt(0))

    clipped_flood = yearly_floods.clip(region)

    flood_check = clipped_flood.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region.geometry(),
        scale=500,
        maxPixels=1e8
    )

    return ee.Feature(None, {'year': year, 'flood_count': flood_check.get('flooded')})

In [19]:
#load the datasets
#temperature - ERA5-Land dataset (temperature in Kelvin)
temp_dataset = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
                .filterDate(start_date, end_date)
                .select('temperature_2m'))

#precipitation - ERA5-Land dataset (precipitation in meters)
precip_dataset = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
                .filterDate(start_date, end_date)
                .select('total_precipitation'))

#floods -  MODIS Global Flood Database (GFD) and JRC permanent water mask
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')
jrc = (ee.ImageCollection('JRC/GSW1_4/YearlyHistory')
       .select('waterClass')
       .map(lambda img: img.eq(3))  # Permanent water class
       .max())

#drought - SPEI dataset
spei_dataset = ee.ImageCollection("CSIC/SPEI/2_10") \
    .filterBounds(region) \
    .filterDate("1992-01-01", "2022-01-01")


#---------------------
#---TEMPERATURE-------
#---------------------

# Calculate the mean, min, and max annual temperature
mean_temp = temp_dataset.mean().clip(region)
min_temp = temp_dataset.min().clip(region)
max_temp = temp_dataset.max().clip(region)

# Convert temperature from Kelvin to Celsius
mean_temp_celsius = mean_temp.subtract(273.15)
min_temp_celsius = min_temp.subtract(273.15)
max_temp_celsius = max_temp.subtract(273.15)

# Calculate the average, min, and max temperature over the region
temp_stats = mean_temp_celsius.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

min_stats = min_temp_celsius.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

max_stats = max_temp_celsius.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Extract and print temperature values
avg_temp = temp_stats.get('temperature_2m').getInfo()
min_temp_value = min_stats.get('temperature_2m').getInfo()
max_temp_value = max_stats.get('temperature_2m').getInfo()

print(f'Average Annual Temperature (°C): {avg_temp}')
print(f'Minimum Annual Temperature (°C): {min_temp_value}')
print(f'Maximum Annual Temperature (°C): {max_temp_value}')

# Define threshold temperature in Celsius
threshold = 32
threshold_k = threshold + 273.15  # Convert to Kelvin

# Apply threshold to identify hot pixels
col_threshold = temp_dataset.map(lambda image: image.gt(threshold_k).set('system:time_start', image.get('system:time_start')))

# Convert to FeatureCollection
days_above_32_fc = ee.FeatureCollection(col_threshold.map(count_hot_days))

# Sum the number of hot hours
total_hot_hours = days_above_32_fc.aggregate_sum('day_above_32')

# Convert hot hours to days
total_days_above_32 = ee.Number(total_hot_hours).divide(24).getInfo()

# Print the results
print(f'Total number of days with at least one pixel above 32°C: {total_days_above_32}')

#------------------------
#-----PRECIPITATION------
#------------------------

# Define the dry and wet seasons
dry_season_1stmonth = 11
dry_season_lastmonth = 5
wet_season_1stmonth = 6
wet_season_lastmonth = 10

# Convert hourly precipitation to daily precipitation (sum over each day)
daily_precip = precip_dataset.map(lambda image: image.multiply(1000).set('date', ee.Date(image.date()).format('YYYY-MM-dd')))

# Compute cumulative annual precipitation
cumulative_annual_precip = daily_precip.sum().clip(region)
cumulative_annual_precip_stats = cumulative_annual_precip.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Compute daily average precipitation
daily_avg_precip_stats = ee.Dictionary(cumulative_annual_precip_stats).map(lambda key, value: ee.Number(value).divide(365))

# Compute cumulative precipitation for wet season
wet_season_precip = precip_dataset.filter(ee.Filter.calendarRange(wet_season_1stmonth, wet_season_lastmonth, 'month')).sum().multiply(1000).clip(region)
wet_season_precip_stats = wet_season_precip.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Compute cumulative precipitation for dry season
dry_season_precip = precip_dataset.filter(ee.Filter.calendarRange(dry_season_1stmonth, dry_season_lastmonth, 'month')).sum().multiply(1000).clip(region)
dry_season_precip_stats = dry_season_precip.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Extract and print precipitation values
cumulative_annual_precip_value = cumulative_annual_precip_stats.get('total_precipitation').getInfo()
daily_avg_precip_value = daily_avg_precip_stats.get('total_precipitation').getInfo()
wet_season_precip_value = wet_season_precip_stats.get('total_precipitation').getInfo()
dry_season_precip_value = dry_season_precip_stats.get('total_precipitation').getInfo()

print(f'Cumulative Annual Precipitation (mm): {cumulative_annual_precip_value}')
print(f'Daily Average Precipitation (mm): {daily_avg_precip_value}')
print(f'Wet Season Cumulative Precipitation (mm): {wet_season_precip_value}')
print(f'Dry Season Cumulative Precipitation (mm): {dry_season_precip_value}')

#---------------------------
#---------FLOODS------------
#---------------------------

# Define years for flood analysis
years = ee.List.sequence(2000, 2018)

# Convert flood counts to FeatureCollection
flood_counts_fc = ee.FeatureCollection(years.map(count_flood_events))

total_floods = flood_counts_fc.aggregate_sum('flood_count').getInfo()

if total_floods > 0:
    print('Floods detected in the project area')
else:
    print('No floods detected in the project area')

print('Number of flood events per year:', flood_counts_fc.getInfo())


#---------------------------
#---------DROUGHT-----------
#---------------------------

# Compute average SPEI indices
spei_avg = spei_dataset.reduce(ee.Reducer.mean()).clip(region).select([
    "SPEI_03_month_mean", "SPEI_06_month_mean", "SPEI_09_month_mean", "SPEI_12_month_mean"
])

# Compute drought risk based on SPEI-9
chart_data = spei_dataset.map(lambda image:
    ee.Feature(None, {
        "Date": image.get("system:time_start"),
        "SPEI_09_month": image.select("SPEI_09_month").reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region.geometry(),
            scale=55660,
            maxPixels=1e9
        ).get("SPEI_09_month")
    })
)

# Convert to FeatureCollection
chart_list = chart_data.toList(chart_data.size())

# Filter drought events (SPEI-9 < -1.5)
drought_events = chart_list.filter(ee.Filter.lt("SPEI_09_month", -1.5))

# Compute drought risk percentage
total_features = chart_list.size()
num_drought_events = drought_events.size()
percentage_drought = ee.Number(num_drought_events).divide(total_features).multiply(100)

# Classify drought risk
risk_level = ee.Algorithms.If(
    percentage_drought.lt(5), "Low risk",
    ee.Algorithms.If(percentage_drought.lt(15), "Medium risk", "High risk")
)

# Print drought risk
print("Percentage of months with severe drought:", percentage_drought.getInfo())
print("Drought Risk Level:", risk_level.getInfo())



Average Annual Temperature (°C): 27.697347194349774
Minimum Annual Temperature (°C): 17.78302917480471
Maximum Annual Temperature (°C): 40.22042236328127
Total number of days with at least one pixel above 32°C: 68.45833333333333
Cumulative Annual Precipitation (mm): 2950147.0467276038
Daily Average Precipitation (mm): 8082.594648568777
Wet Season Cumulative Precipitation (mm): 2400707.7876665276
Dry Season Cumulative Precipitation (mm): 549439.2590610355
No floods detected in the project area
Number of flood events per year: {'type': 'FeatureCollection', 'columns': {'flood_count': 'Float', 'system:index': 'String', 'year': 'Float'}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '0', 'properties': {'flood_count': 0, 'year': 2000}}, {'type': 'Feature', 'geometry': None, 'id': '1', 'properties': {'flood_count': 0, 'year': 2001}}, {'type': 'Feature', 'geometry': None, 'id': '2', 'properties': {'flood_count': 0, 'year': 2002}}, {'type': 'Feature', 'geometry': None, 'id': '3', 'pr

In [11]:
# Load the ERA5-Land dataset (temperature in Kelvin)
temp_dataset = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
                .filterDate(start_date, end_date)
                .select('temperature_2m'))

# Calculate the mean, min, and max annual temperature
mean_temp = temp_dataset.mean().clip(region)
min_temp = temp_dataset.min().clip(region)
max_temp = temp_dataset.max().clip(region)

# Convert temperature from Kelvin to Celsius
mean_temp_celsius = mean_temp.subtract(273.15)
min_temp_celsius = min_temp.subtract(273.15)
max_temp_celsius = max_temp.subtract(273.15)

# Calculate the average, min, and max temperature over the region
temp_stats = mean_temp_celsius.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

min_stats = min_temp_celsius.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

max_stats = max_temp_celsius.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Extract and print temperature values
avg_temp = temp_stats.get('temperature_2m').getInfo()
min_temp_value = min_stats.get('temperature_2m').getInfo()
max_temp_value = max_stats.get('temperature_2m').getInfo()

print(f'Average Annual Temperature (°C): {avg_temp}')
print(f'Minimum Annual Temperature (°C): {min_temp_value}')
print(f'Maximum Annual Temperature (°C): {max_temp_value}')

# Define threshold temperature in Celsius
threshold = 32
threshold_k = threshold + 273.15  # Convert to Kelvin

# Apply threshold to identify hot pixels
col_threshold = temp_dataset.map(lambda image: image.gt(threshold_k).set('system:time_start', image.get('system:time_start')))

# Count the number of days with at least one pixel exceeding the threshold
def count_hot_days(image):
    mask = image.reduceRegion(
        reducer=ee.Reducer.anyNonZero(),
        geometry=region,
        scale=1000,
        maxPixels=1e8
    )
    is_above = ee.Algorithms.If(mask.get('temperature_2m'), 1, 0)
    return ee.Feature(None, {'date': image.get('system:time_start'), 'day_above_32': is_above})

# Convert to FeatureCollection
days_above_32_fc = ee.FeatureCollection(col_threshold.map(count_hot_days))

# Sum the number of hot hours
total_hot_hours = days_above_32_fc.aggregate_sum('day_above_32')

# Convert hot hours to days
total_days_above_32 = ee.Number(total_hot_hours).divide(24).getInfo()

# Print the results
print(f'Total number of days with at least one pixel above 32°C: {total_days_above_32}')

Average Annual Temperature (°C): 27.697347194349774
Minimum Annual Temperature (°C): 17.78302917480471
Maximum Annual Temperature (°C): 40.22042236328127
Total number of days with at least one pixel above 32°C: 68.45833333333333


In [12]:
# Define the dry and wet seasons
dry_season_1stmonth = 11
dry_season_lastmonth = 5
wet_season_1stmonth = 6
wet_season_lastmonth = 10

# Load the ERA5-Land dataset (precipitation in meters)
precip_dataset = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
                .filterDate(start_date, end_date)
                .select('total_precipitation'))

# Convert hourly precipitation to daily precipitation (sum over each day)
daily_precip = precip_dataset.map(lambda image: image.multiply(1000).set('date', ee.Date(image.date()).format('YYYY-MM-dd')))

# Compute cumulative annual precipitation
cumulative_annual_precip = daily_precip.sum().clip(region)
cumulative_annual_precip_stats = cumulative_annual_precip.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Compute daily average precipitation
daily_avg_precip_stats = ee.Dictionary(cumulative_annual_precip_stats).map(lambda key, value: ee.Number(value).divide(365))

# Compute cumulative precipitation for wet season
wet_season_precip = precip_dataset.filter(ee.Filter.calendarRange(wet_season_1stmonth, wet_season_lastmonth, 'month')).sum().multiply(1000).clip(region)
wet_season_precip_stats = wet_season_precip.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Compute cumulative precipitation for dry season
dry_season_precip = precip_dataset.filter(ee.Filter.calendarRange(dry_season_1stmonth, dry_season_lastmonth, 'month')).sum().multiply(1000).clip(region)
dry_season_precip_stats = dry_season_precip.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=region.geometry(),
    scale=1000,
    bestEffort=True
)

# Extract and print precipitation values
cumulative_annual_precip_value = cumulative_annual_precip_stats.get('total_precipitation').getInfo()
daily_avg_precip_value = daily_avg_precip_stats.get('total_precipitation').getInfo()
wet_season_precip_value = wet_season_precip_stats.get('total_precipitation').getInfo()
dry_season_precip_value = dry_season_precip_stats.get('total_precipitation').getInfo()

print(f'Cumulative Annual Precipitation (mm): {cumulative_annual_precip_value}')
print(f'Daily Average Precipitation (mm): {daily_avg_precip_value}')
print(f'Wet Season Cumulative Precipitation (mm): {wet_season_precip_value}')
print(f'Dry Season Cumulative Precipitation (mm): {dry_season_precip_value}')

Cumulative Annual Precipitation (mm): 2950147.0467276038
Daily Average Precipitation (mm): 8082.594648568777
Wet Season Cumulative Precipitation (mm): 2400707.7876665276
Dry Season Cumulative Precipitation (mm): 549439.2590610355


In [13]:
# Load flood event data from the MODIS Global Flood Database (GFD)
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')

# Load JRC permanent water mask
jrc = (ee.ImageCollection('JRC/GSW1_4/YearlyHistory')
       .select('waterClass')
       .map(lambda img: img.eq(3))  # Permanent water class
       .max())

# Define years for flood analysis
years = ee.List.sequence(2000, 2018)

def count_flood_events(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)

    yearly_floods = (gfd.filterDate(start_date, end_date)
                      .select('flooded')
                      .map(lambda img: img.gt(0).And(jrc.Not()))
                      .sum()
                      .gt(0))

    clipped_flood = yearly_floods.clip(region)

    flood_check = clipped_flood.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region.geometry(),
        scale=500,
        maxPixels=1e8
    )

    return ee.Feature(None, {'year': year, 'flood_count': flood_check.get('flooded')})

# Convert flood counts to FeatureCollection
flood_counts_fc = ee.FeatureCollection(years.map(count_flood_events))

total_floods = flood_counts_fc.aggregate_sum('flood_count').getInfo()

if total_floods > 0:
    print('Floods detected in the project area')
else:
    print('No floods detected in the project area')

print('Number of flood events per year:', flood_counts_fc.getInfo())


No floods detected in the project area
Number of flood events per year: {'type': 'FeatureCollection', 'columns': {'flood_count': 'Float', 'system:index': 'String', 'year': 'Float'}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '0', 'properties': {'flood_count': 0, 'year': 2000}}, {'type': 'Feature', 'geometry': None, 'id': '1', 'properties': {'flood_count': 0, 'year': 2001}}, {'type': 'Feature', 'geometry': None, 'id': '2', 'properties': {'flood_count': 0, 'year': 2002}}, {'type': 'Feature', 'geometry': None, 'id': '3', 'properties': {'flood_count': 0, 'year': 2003}}, {'type': 'Feature', 'geometry': None, 'id': '4', 'properties': {'flood_count': 0, 'year': 2004}}, {'type': 'Feature', 'geometry': None, 'id': '5', 'properties': {'flood_count': 0, 'year': 2005}}, {'type': 'Feature', 'geometry': None, 'id': '6', 'properties': {'flood_count': 0, 'year': 2006}}, {'type': 'Feature', 'geometry': None, 'id': '7', 'properties': {'flood_count': 0, 'year': 2007}}, {'type': 'Feature', '

In [15]:
# Load SPEI dataset
spei_dataset = ee.ImageCollection("CSIC/SPEI/2_10") \
    .filterBounds(region) \
    .filterDate("1992-01-01", "2022-01-01")

# Compute average SPEI indices
spei_avg = spei_dataset.reduce(ee.Reducer.mean()).clip(region).select([
    "SPEI_03_month_mean", "SPEI_06_month_mean", "SPEI_09_month_mean", "SPEI_12_month_mean"
])

# Compute drought risk based on SPEI-9
chart_data = spei_dataset.map(lambda image:
    ee.Feature(None, {
        "Date": image.get("system:time_start"),
        "SPEI_09_month": image.select("SPEI_09_month").reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region.geometry(),
            scale=55660,
            maxPixels=1e9
        ).get("SPEI_09_month")
    })
)

# Convert to FeatureCollection
chart_list = chart_data.toList(chart_data.size())

# Filter drought events (SPEI-9 < -1.5)
drought_events = chart_list.filter(ee.Filter.lt("SPEI_09_month", -1.5))

# Compute drought risk percentage
total_features = chart_list.size()
num_drought_events = drought_events.size()
percentage_drought = ee.Number(num_drought_events).divide(total_features).multiply(100)

# Classify drought risk
risk_level = ee.Algorithms.If(
    percentage_drought.lt(5), "Low risk",
    ee.Algorithms.If(percentage_drought.lt(15), "Medium risk", "High risk")
)

# Print drought risk
print("Percentage of months with severe drought:", percentage_drought.getInfo())
print("Drought Risk Level:", risk_level.getInfo())


Percentage of months with severe drought: 9.166666666666666
Drought Risk Level: Medium risk
